In [1]:
import joblib
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
import requests
import json

# Load Posenet

In [2]:
interpreter = tf.lite.Interpreter("C:/Users/hjhhi/Desktop/Codes/lite-model_movenet_singlepose_lightning_tflite_float16_4.tflite")
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
interpreter.allocate_tensors()
output = interpreter.get_output_details()[0]  # Model has single output.
input = interpreter.get_input_details()[0]  # Model has single input.
interpreter.invoke()

In [3]:
def image_preprocess(path):
    image = cv2.imread(path)
    image= image.astype("uint8")
    image = cv2.resize(image, (192, 192), interpolation=cv2.INTER_NEAREST)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image= np.resize(image, (1, 192, 192, 3))
    return image

# Load data from Mobius

## Load image data   
이 과정은 모비우스에서 가져오기 +  base64 디코딩 과정을 거쳐야함

In [4]:
image = image_preprocess("C:/Users/hjhhi/Desktop/Codes/51.jpg")

interpreter.set_tensor(input['index'], image)
interpreter.invoke()
result = interpreter.get_tensor(output_details[0]['index'])
result = result.reshape(51)

## Load watch data 

In [29]:
url = "http://203.253.128.177:7579/Mobius/SW_Hackaton/watch/la"

payload=""
headers = {
'Accept': 'application/json',
'X-M2M-RI': '12345',
'X-M2M-Origin': 'S',
'Content-Type': 'application/json'
}

response = requests.request("GET", url, headers=headers, data=payload)
gyro_data = json.loads(response.text)["m2m:cin"]["con"]
gyro_data = gyro_data.split(",")

print("Gyro data value is: ", gyro_data)

Gyro data value is:  ['-0.017104227', '-0.0012217305', '0.02076942', '0.59136784', '8.0684595', '5.2289367']


# Load classifier 

## Load image classifier

In [7]:
image_classifier = joblib.load("./image_model.pkl")

In [8]:
image_proba = image_classifier.predict_proba([result])

In [9]:
image_proba

array([[0.01758501, 0.00950702, 0.57819063, 0.12256325, 0.05857313,
        0.08835219, 0.12522878]])

## Load gyro classifier 

In [32]:
gyro_classifier = joblib.load("./gyro_model.pkl")

In [33]:
gyro_data

['-0.017104227',
 '-0.0012217305',
 '0.02076942',
 '0.59136784',
 '8.0684595',
 '5.2289367']

In [34]:
gyro_proba = gyro_classifier.predict_proba([gyro_data])

C:\Users\hjhhi\Anaconda3\envs\raspberry\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [35]:
gyro_proba

array([[0.        , 0.05559563, 0.62373721, 0.18899256, 0.10696308,
        0.0193828 , 0.00532873]])

# Send final prediction to Mobius 

In [40]:
Final_proba = image_proba + gyro_proba

In [41]:
Final_pred=np.argmax(Final_proba)

In [45]:
import requests
import json

url = "http://203.253.128.177:7579/Mobius/SW_Hackaton/final_pred"

data = {
    "m2m:cin": {
        "con": float(Final_pred
    }
}

headers = {
  'Accept': 'application/json',
  'X-M2M-RI': '12345',
  'X-M2M-Origin': 'SOrigin',
  'Content-Type': 'application/json;ty=4'
}



response = requests.request("POST", url, headers=headers, data=json.dumps(data))

print(response.text)

TypeError: Object of type int64 is not JSON serializable